# 0 Outline
1. Data Process Functions or Class, APIs:
    - can input from excel or others.
    - Output 2-D data points.
        - 1th dimention indicates single data point(x,y,z,t,value).
        - 2th dimention indicates the number of data points.
    - Every data point has form (x,y,z,t,value), where x,y,z is spatial corordinate, t is temporal corordinates, and value is function(to be solved) value.
    - be Tensor.
    - can assign accuracy for each number.
    - can assign whether gradient 'on' or 'off'.

2. Model Functions or Class, APIs:
    - models
3. Tool Functions or Class, APIs:
    - train
    - evaulate
4. Visualization Functions or Class, APIs:


关于数据集的一些信息
- **似乎**matlab产生的数据集都是边界点在前。
    - 二维圆盘热方程为例，前56项都是边界的mesh。

方程形式：
$$
\begin{aligned}
\begin{cases}
u_t(x,t) &= \triangle u(x,t),\quad t\in[0,2],\quad x\in\Omega = \{|x|^2 \leq 1\} \\
\frac{\partial u(x,t)}{\partial \mathbf{n}} &= u(x,t)^4 - u_0^4,\quad x\in \partial\Omega \\
u(x,0) &= c.
\end{cases}
\end{aligned}
$$


# import modules involved

In [30]:
import torch
import pandas
import torch.nn as nn
import collections
import numpy as np

import itertools

In [31]:
x = [(1,2),(2,3)]
y = [1,2]
list(itertools.product(x, y))

[((1, 2), 1), ((1, 2), 2), ((2, 3), 1), ((2, 3), 2)]

# Data Functions or Classes.

In [32]:
class DataProcess:
    """
    数据读取与预处理
    主要是数据读取
    流程(以Excel为例)
    1.读取Excel表 (需要Pandas包)
    2.把数据转换成Tensor格式
    3.把数据变成指定形状
        - 二维矩阵
        - 行数代表数据点个数
        - 每一行代表一个数据点，(x,y,z,t,value)
    4.其它要求
        - 区分边界点和内部点
        - 区分训练集和验证集
    """
    def __init__(self,input,output,spatial_dimension):
        pass

    def ReadFromExcel(self,filename):
        pass

# Model Functions or Classes
- model

In [33]:
class HeatEqModel(nn.Module):
    """
    - 确定深度模型
    - 前向函数
    - 需要需要确定方程的系数
    """
    def __init__(
        self,
        input_size,
        hidden_size,
        output_size,
        depth,
        mass_density,
        specific_heat,
        thermal_conductivity,
        heat_source,
        emissivity,
        Stefan_Boltzmann_constant,
        AmbientTemperature,
        act=torch.nn.Tanh
    ):
        super(HeatEqModel, self).__init__()
        
        self.mass_density = mass_density
        self.specific_heat = specific_heat
        self.thermal_conductivity = thermal_conductivity
        self.heat_source = heat_source
        self.emissivity = emissivity
        self.Stefan_Boltzmann_constant = Stefan_Boltzmann_constant
        self.AmbientTemperature = AmbientTemperature

        layers = [('input', torch.nn.Linear(input_size, hidden_size))]
        layers.append(('input_activation', act()))
        for i in range(depth): 
            layers.append(
                ('hidden_%d' % i, torch.nn.Linear(hidden_size, hidden_size))
            )
            layers.append(('activation_%d' % i, act()))
        layers.append(('output', torch.nn.Linear(hidden_size, output_size)))

        layerDict = collections.OrderedDict(layers)
        self.layers = torch.nn.Sequential(layerDict)

    def forward(self, x):
        out = self.layers(x)
        return out

# Util Functions or Classes
- train
- evalute

In [34]:
torch.cuda.is_available()

True

In [45]:
class Util:
    """
    包括但不限于：
    - 数据的处理过程
      - 数据类型 long float
      - 数据存储位置cpu or gpu
    - 模型的实例化
    - 定义损失函数
    - 定义训练过程
    - 定义evaluate过程
    - 
    """
    def __init__(self,xy_boundary,xy_in,t,Data = None):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.device = device
        self.xy_boudnary = xy_boundary.to(device) # 边界点在前。
        self.xy_in = xy_in.to(device)
        self.xy = torch.cat((xy_boundary,xy_in),dim=0).to(device)
        self.t = t.to(device) # 从小到大。2D
        self.data = Data # Data可以视作一张表。

        self.model = HeatEqModel(
            input_size=3,
            hidden_size=20,
            output_size=1,
            depth=4,
            mass_density=7.87,
            specific_heat=0.44,
            thermal_conductivity=76.2,
            heat_source=0.,
            emissivity=0.1,
            Stefan_Boltzmann_constant=5.6703*10**(-8),
            AmbientTemperature = 300.,
            act=torch.nn.Tanh
        ).to(device)
        
        # 处理数据
        # 边界值点，空间坐标和时间坐标合并成一个二维张量。
        self.boundary = self.xy_boudnary.detach().clone().to(device)
        self.t_boundary = torch.full((self.xy_boudnary.shape[0],1),self.t[0][0].item()).to(device)
        for i in range(self.t.shape[0] - 1):
            self.boundary = torch.cat((self.boundary,self.xy_boudnary),dim=0)
            ti = torch.full((self.xy_boudnary.shape[0],1),self.t[i+1][0].item()).to(torch.float).to(device)
            self.t_boundary = torch.cat((self.t_boundary,ti),dim=0)

        self.AmbientTemperature_boundary = torch.full((self.boundary.shape[0],1),
                                                      self.model.AmbientTemperature).to(torch.float).to(device)

        self.boundary = torch.cat((self.boundary,self.t_boundary),dim = 1)


        # 初值点，
        self.initial = self.xy.detach().clone().to(device)
        self.t0 = torch.full((self.xy.shape[0],1),0).to(device)

        self.initial = torch.cat((self.initial,self.t0),dim=1)
        self.initial.requires_grad_()

        # 内部点，
        # 先把self.xy,self.t变成1D.
        # 再用itertools.product求卡式积。
        self.In = self.xy_in.detach().clone()
        t_temporal = torch.full((self.xy_in.shape[0],1),self.t[0][0].item()).to(device)
        for i in range(self.t.shape[0] - 1):
            t_temporal = torch.cat((t_temporal,
                                   torch.full((self.xy_in.shape[0],1),self.t[i+1][0].item()).to(device)),
                                   dim = 0).to(device)
            self.In = torch.cat((self.In,self.xy_in.detach().clone()),dim=0).to(device)
        self.In = torch.cat((self.In,t_temporal),dim = 1).to(device)
        self.In.requires_grad_()
        
        self.criterion = torch.nn.MSELoss()
        self.iter = 1
        
        self.optimizer = torch.optim.LBFGS(
            self.model.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps
        )
        
        self.adam = torch.optim.Adam(self.model.parameters())
        
    def loss_func(self):
        self.adam.zero_grad()
        self.optimizer.zero_grad()

        # 初边值条件产生的loss_data = loss_boundary + loss_ini.

        # loss_boundary MSE()($\triangle T \cdot \vector{x} + \epsilon \sigma (T^4 - T_{\infty}^4)$ - 0).        
        y_predBoudnary = self.model(self.boundary)

        dy_predBoundary_dx = torch.autograd.grad(inputs=self.boundary, outputs=y_predBoudnary,
                                                 grad_outputs=torch.ones_like(y_predBoudnary),
                                                 retain_graph=True,
                                                 create_graph=True)[0][:,0].unsqueeze(1)
        
        dy_predBoundary_dy = torch.autograd.grad(inputs=self.boundary, outputs=y_predBoudnary,
                                                 grad_outputs=torch.ones_like(y_predBoudnary),
                                                 retain_graph=True,
                                                 create_graph=True)[0][:,1].unsqueeze(1)
        
        Nabla_y_predBoundary = torch.cat((dy_predBoundary_dx,dy_predBoundary_dy),
                                         dim = 1)
        
        NormalVector = self.boundary[:,0:2]
        #print(f"Nabla_y_predBoundary{Nabla_y_predBoundary}")
        #print(f"NormalVector{NormalVector}")
        dy_dn = torch.sum(Nabla_y_predBoundary * NormalVector,dim=1).unsqueeze(1)

        #print("\nloss_boundary:\n")
        #print(f"self.model.Stefan_Boltzmann_constant * self.model.emissivity * self.AmbientTemperature_boundary**4{self.model.Stefan_Boltzmann_constant * self.model.emissivity * self.AmbientTemperature_boundary**4}")
        #print(f"dy_dn + y_predBoudnary**4 * self.model.Stefan_Boltzmann_constant * self.model.emissivity{dy_dn + y_predBoudnary**4 * self.model.Stefan_Boltzmann_constant * self.model.emissivity}")
        #print(f"dy_dn{dy_dn}")
        #print(f"y_predBoudnary{y_predBoudnary}")
        #print(f"self.model.Stefan_Boltzmann_constant{self.model.Stefan_Boltzmann_constant}")
        #print(f"self.model.emissivity{self.model.emissivity}")
        # loss_boundary MSE()($\triangle T \cdot \vector{x} + \epsilon \sigma (T^4 - T_{\infty}^4)$ - 0).Sefan_Boltzmann_constant
        loss_boundary = self.criterion(
                        self.model.Stefan_Boltzmann_constant * self.model.emissivity * self.AmbientTemperature_boundary**4,
                        dy_dn + y_predBoudnary**4 * self.model.Stefan_Boltzmann_constant * self.model.emissivity
                        )
        
        y_predIni = self.model(self.initial)

        # loss_ini = MSE()(y_predIni,0)
        #print("\n loss_ini\n")
        #print(f"y_predIni{y_predIni}")
        #print(f"torch.full((y_predIni.shape[0],1),0).to(torch.float32){torch.full((y_predIni.shape[0],1),0).to(torch.float32)}")
        loss_ini = self.criterion(y_predIni,torch.full((y_predIni.shape[0],1),0).to(torch.float32).to(self.device))


        loss_data = loss_ini + loss_boundary

        # loss_pde = MSE()(\rho c \frac{\partial T}{\partial t} - k\triangle T = Q, 0).
        y_predIn = self.model(self.In)


        dy_predIn_dx = torch.autograd.grad(inputs=self.In, outputs=y_predIn,
                                                 grad_outputs=torch.ones_like(y_predIn),
                                                 retain_graph=True,
                                                 create_graph=True)[0][:,0].unsqueeze(1)
        
        dy_predIn_dy = torch.autograd.grad(inputs=self.In, outputs=y_predIn,
                                                 grad_outputs=torch.ones_like(y_predIn),
                                                 retain_graph=True,
                                                 create_graph=True)[0][:,1].unsqueeze(1)
        dy_predIn_dt = torch.autograd.grad(inputs=self.In, outputs=y_predIn,
                                                 grad_outputs=torch.ones_like(y_predIn),
                                                 retain_graph=True,
                                                 create_graph=True)[0][:,2].unsqueeze(1)

        dy_predIn_dxdx = torch.autograd.grad(inputs=self.In, outputs=dy_predIn_dx,
                                                 grad_outputs=torch.ones_like(dy_predIn_dx),
                                                 retain_graph=True,
                                                 create_graph=True)[0][:,0].unsqueeze(1)
        
        dy_predIn_dydy = torch.autograd.grad(inputs=self.In, outputs=dy_predIn_dy,
                                                 grad_outputs=torch.ones_like(dy_predIn_dy),
                                                 retain_graph=True,
                                                 create_graph=True)[0][:,1].unsqueeze(1)
        
        #print("\n loss pde \n")
        #print(f"dy_predIn_dt*self.model.specific_heat*self.model.mass_density{dy_predIn_dt*self.model.specific_heat*self.model.mass_density}")
        #print(f"self.model.thermal_conductivity*(dy_predIn_dxdx + dy_predIn_dydy{self.model.thermal_conductivity*(dy_predIn_dxdx + dy_predIn_dydy)}")
        loss_pde = self.criterion(dy_predIn_dt*self.model.specific_heat*self.model.mass_density,
                                  self.model.thermal_conductivity*(dy_predIn_dxdx + dy_predIn_dydy)
                                  )

        loss = loss_pde + loss_data
        loss.backward()


        if self.iter % 100 == 0: 
            print(self.iter, loss.item())
        self.iter = self.iter + 1
        return loss
    
    def train(self):
        for i in range(30000):
            self.adam.step(self.loss_func)
        self.optimizer.step(self.loss_func)
    
    def eval_(self):
        self.model.eval()

In [36]:
io = "..\graduate\matlab_heatEq\二维圆盘数据.xlsx"
data = pandas.read_excel(io)

In [37]:
xy = pandas.DataFrame(data,columns = ['x','y'])
t = np.arange(0,2.01,0.01)
tlist = np.expand_dims(t, axis=1)

In [38]:
xy = xy.to_numpy()

In [39]:
xy_boundary = xy[0:56,0:2]
xy_in = xy[56::,0:2]

In [40]:
t_tensor = torch.from_numpy(tlist)
xy_boundary_tensor = torch.from_numpy(xy_boundary)
xy_in_tensor = torch.from_numpy(xy_in)
t_tensor.requires_grad_()
xy_in_tensor.requires_grad_()
xy_boundary_tensor.requires_grad_()

tensor([[-1.0000e+00, -1.2246e-16],
        [ 6.1232e-17, -1.0000e+00],
        [ 1.0000e+00,  0.0000e+00],
        [-1.8370e-16,  1.0000e+00],
        [-9.9370e-01, -1.1210e-01],
        [-9.7490e-01, -2.2265e-01],
        [-9.4387e-01, -3.3033e-01],
        [-9.0099e-01, -4.3384e-01],
        [-8.4680e-01, -5.3192e-01],
        [-7.8191e-01, -6.2339e-01],
        [-7.0711e-01, -7.0711e-01],
        [-6.2339e-01, -7.8191e-01],
        [-5.3192e-01, -8.4680e-01],
        [-4.3384e-01, -9.0099e-01],
        [-3.3033e-01, -9.4387e-01],
        [-2.2265e-01, -9.7490e-01],
        [-1.1210e-01, -9.9370e-01],
        [-1.1210e-01,  9.9370e-01],
        [-2.2265e-01,  9.7490e-01],
        [-3.3033e-01,  9.4387e-01],
        [-4.3384e-01,  9.0099e-01],
        [-5.3192e-01,  8.4680e-01],
        [-6.2339e-01,  7.8191e-01],
        [-7.0711e-01,  7.0711e-01],
        [-7.8191e-01,  6.2339e-01],
        [-8.4680e-01,  5.3192e-01],
        [-9.0099e-01,  4.3384e-01],
        [-9.4387e-01,  3.303

In [41]:
t_tensor.shape

torch.Size([201, 1])

In [46]:
net = Util(xy_boundary = xy_boundary_tensor.to(torch.float32)
           ,xy_in = xy_in_tensor.to(torch.float32)
           ,t = t_tensor.to(torch.float32))
net.train()

100 2109.3623046875
200 2104.941650390625
300 2101.081298828125
400 2098.17724609375
500 2095.4169921875
600 2092.912353515625
700 2090.495849609375
800 2088.434814453125
900 2085.985595703125
1000 2083.652587890625
1100 2081.482177734375
1200 2079.415771484375
1300 2076.938232421875
1400 2074.7216796875
1500 2076.53173828125
1600 2070.18994140625
1700 2074.620361328125
1800 2065.556396484375
1900 2065.746337890625
2000 2060.768310546875
2100 2058.3525390625
2200 2055.775634765625
2300 2066.603271484375
2400 2050.52099609375
2500 2047.781494140625
2600 2044.94091796875
2700 2041.914306640625
2800 2038.9580078125
2900 2035.6829833984375
3000 2032.5048828125
3100 2028.8973388671875
3200 2026.2471923828125
3300 2021.541259765625
3400 2017.3861083984375
3500 2013.3428955078125
3600 2008.7081298828125
3700 2004.939453125
3800 1999.00146484375
3900 1995.3603515625
4000 1988.043212890625
4100 1983.0318603515625
4200 1975.7100830078125
4300 1968.733642578125
4400 1961.803466796875
4500 1953.93

In [44]:
print(net.loss_func())

tensor(16.8736, device='cuda:0', grad_fn=<AddBackward0>)


In [43]:
# 测试 测试
## 测试数据

#Data t = 0, 0.2, 0.4, 0.6, 0.8, 1
#(0,1)    20 
#(0,-1)
#(-1,0)
#(1,0)
#(0,0)
#(1/2,1/2)
#(1/2,-1/2)
#(-1/2,1/2)
#(1/3,1/4)

#t = torch.tensor([[0.0],[0.2],[0.4],[0.6],[0.8],[1.0]],dtype = torch.float,requires_grad = True)
#xy_boundary = torch.tensor([[0.,1.],[0.,-1.],[-1.,0.],[1.,0.]],dtype = torch.float,requires_grad = True)
#xy_in = torch.tensor([[0.,0.],[0.5,0.5],[0.5,-0.5],[-0.5,0.5],[0.33,0.25]],dtype = torch.float,requires_grad = True)


#net = Util(xy_boundary = xy_boundary,xy_in = xy_in,t = t)
#net.train()